# Language modelling

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

2. Download three Polish models from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples. You can also try using Bielik for that, but make sure you are using the model via Transformers API, not GUI.

In [2]:
pipe_herbert = pipeline("fill-mask", model="allegro/herbert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
pipe_twhin = pipeline("fill-mask", model="Twitter/twhin-bert-base")

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
pipe_trelbert = pipeline("fill-mask", model="deepsense-ai/trelbert")

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


3. Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [19]:
def test_polish_grammatical_cases(unmasker, model_name="Model"):
    test_sentences = {
        "Nominative": "Warszawa to największe <mask> w Polsce.",
        "Genitive": "Nie mam żadnego <mask> do zaproponowania.",
        "Dative": "Przyglądam się nowemu <mask> w parku.",
        "Accusative": "Znalazłem interesujący <mask> w bibliotece.",
        "Instrumental": "Jestem zadowolony z pracy nad tym <mask>.",
        "Locative": "Myślę o spotkaniu na <mask> w kawiarni.",
        "Vocative": "Droga <mask>, chciałbym cię zaprosić na przyjęcie."
    }

    print(f"\nTesting Polish grammatical cases with {model_name}:\n" + "-" * 50)
    for case, sentence in test_sentences.items():
        print(f"{case} Case: {sentence}")
        try:
            predictions = unmasker(sentence)
            top_predictions = [pred['token_str'] for pred in predictions[:3]]
            print("Top Predictions:", top_predictions)
        except Exception as e:
            print(f"Error with {case} case: {e}")
        print()


test_polish_grammatical_cases(pipe_herbert, model_name="HerBERT")
test_polish_grammatical_cases(pipe_twhin, model_name="TwHIN BERT")
test_polish_grammatical_cases(pipe_trelbert, model_name="Polish BERT (dkleczek)")


Testing Polish grammatical cases with HerBERT:
--------------------------------------------------
Nominative Case: Warszawa to największe <mask> w Polsce.
Top Predictions: ['miasto', 'lotnisko', 'centrum']

Genitive Case: Nie mam żadnego <mask> do zaproponowania.
Top Predictions: ['innego', 'więcej', 'nic']

Dative Case: Przyglądam się nowemu <mask> w parku.
Top Predictions: ['budynku', 'muzeum', 'parku']

Accusative Case: Znalazłem interesujący <mask> w bibliotece.
Top Predictions: ['artykuł', 'przedmiot', 'produkt']

Instrumental Case: Jestem zadowolony z pracy nad tym <mask>.
Top Predictions: ['projektem', 'dokumentem', 'programem']

Locative Case: Myślę o spotkaniu na <mask> w kawiarni.
Top Predictions: ['żywo', 'pewno', 'spotkaniu']

Vocative Case: Droga <mask>, chciałbym cię zaprosić na przyjęcie.
Top Predictions: ['Pani', 'pani', 'Elżbieta']


Testing Polish grammatical cases with TwHIN BERT:
--------------------------------------------------
Nominative Case: Warszawa to najwię

4. Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [21]:
test_sentences_gender = {
    "Sentence 1": "Maria poszła do sklepu i <mask> na zakupy.",
    "Sentence 2": "Jan zjadł obiad i <mask> na spacer.",
    "Sentence 3": "Ona wstała rano i <mask> kawę.",
}

def test_gender_consistency(unmasker, model_name="Model"):
    print(f"\nTesting gender consistency with {model_name}:\n" + "-" * 50)

    for description, sentence in test_sentences_gender.items():
        print(f"{description}: {sentence}")
        try:
            predictions = unmasker(sentence)
            top_predictions = [pred['token_str'] for pred in predictions[:3]]
            print("Top Predictions:", top_predictions)
        except Exception as e:
            print(f"Error in processing '{sentence}': {e}")
        print()

test_gender_consistency(pipe_herbert, model_name="HerBERT")
test_gender_consistency(pipe_twhin, model_name="TwHIN BERT")
test_gender_consistency(pipe_trelbert, model_name="Polish BERT (dkleczek)")


Testing gender consistency with HerBERT:
--------------------------------------------------
Sentence 1: Maria poszła do sklepu i <mask> na zakupy.
Top Predictions: ['poszła', 'wyszła', 'pojechała']

Sentence 2: Jan zjadł obiad i <mask> na spacer.
Top Predictions: ['poszedł', 'wyszedł', 'ruszył']

Sentence 3: Ona wstała rano i <mask> kawę.
Top Predictions: ['piła', 'zrobiła', 'przyniosła']


Testing gender consistency with TwHIN BERT:
--------------------------------------------------
Sentence 1: Maria poszła do sklepu i <mask> na zakupy.
Top Predictions: ['idzie', 'ide', 'była']

Sentence 2: Jan zjadł obiad i <mask> na spacer.
Top Predictions: ['ide', 'idzie', 'teraz']

Sentence 3: Ona wstała rano i <mask> kawę.
Top Predictions: ['pije', 'robi', 'zrobił']


Testing gender consistency with Polish BERT (dkleczek):
--------------------------------------------------
Sentence 1: Maria poszła do sklepu i <mask> na zakupy.
Top Predictions: ['poszła', 'pojechała', 'idzie']

Sentence 2: Jan zj

5. Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [23]:
def test_real_world_knowledge(unmasker, model_name="Model"):
    test_sentences_knowledge = {
        "Boiling and Freezing Point": "<mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
        "Central Star": "<mask> jest centralną gwiazdą Układu Słonecznego, wokół której krążą planety.",
        "Capital of Poland": "<mask> jest krajem, którego stolicą jest Warszawa."
    }

    print(f"\nTesting real-world knowledge with {model_name}:\n" + "-" * 50)

    for description, sentence in test_sentences_knowledge.items():
        print(f"{description}: {sentence}")
        try:
            predictions = unmasker(sentence)
            top_predictions = [pred['token_str'] for pred in predictions[:3]]
            print("Top Predictions:", top_predictions)
        except Exception as e:
            print(f"Error with '{sentence}': {e}")
        print()

test_real_world_knowledge(pipe_herbert, model_name="HerBERT")
test_real_world_knowledge(pipe_twhin, model_name="TwHIN BERT")
test_real_world_knowledge(pipe_trelbert, model_name="Polish BERT (dkleczek)")


Testing real-world knowledge with HerBERT:
--------------------------------------------------
Boiling and Freezing Point: <mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Top Predictions: ['Woda', 'Słońce', 'Ziemia']

Central Star: <mask> jest centralną gwiazdą Układu Słonecznego, wokół której krążą planety.
Top Predictions: ['Słońce', 'Ziemia', 'Księżyc']

Capital of Poland: <mask> jest krajem, którego stolicą jest Warszawa.
Top Predictions: ['Polska', 'Ukraina', 'Białoruś']


Testing real-world knowledge with TwHIN BERT:
--------------------------------------------------
Boiling and Freezing Point: <mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Top Predictions: ['Ukraina', 'Putin', 'Polska']

Central Star: <mask> jest centralną gwiazdą Układu Słonecznego, wokół której krążą planety.
Top Predictions: ['Polska', 'Bóg', 'Twitter']

Capital of Poland: <mask> jest krajem, którego stolicą jest Warszawa.
Top Prediction

6. Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

In [25]:
def test_zero_shot_sentiment(unmasker, model_name="Model"):
    test_sentences_sentiment = [
        "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta jest zdecydowanie <mask>.",
        "'Ten film był absolutną katastrofą. Nigdy więcej go nie obejrzę.' Wypowiedź ta jest zdecydowanie <mask>.",
        "'Ten film był całkiem ok, ani za dobry, ani za słaby.' Wypowiedź ta jest zdecydowanie <mask>.",
        "'To najlepszy film, jaki kiedykolwiek widziałem! Zdecydowanie polecam.' Wypowiedź ta jest zdecydowanie <mask>.",
        "'Spodziewałem się więcej. Film nie spełnił moich oczekiwań.' Wypowiedź ta jest zdecydowanie <mask>."
    ]

    prompt_variations = [
        "Wypowiedź ta jest zdecydowanie <mask>.",
        "Ta wypowiedź jest raczej <mask>, prawda?",
        "Można powiedzieć, że ta wypowiedź jest <mask>.",
        "Według mnie, ta opinia jest <mask>.",
        "Oceniam tę wypowiedź jako <mask>."
    ]

    print(f"\nTesting zero-shot sentiment analysis with {model_name}:\n" + "-" * 50)

    for sentence in test_sentences_sentiment:
        print(f"\nOriginal Sentence: {sentence}")

        for prompt in prompt_variations:
            test_sentence = sentence.replace("Wypowiedź ta jest zdecydowanie <mask>.", prompt)
            print(f"Prompt Variation: {test_sentence}")

            try:
                predictions = unmasker(test_sentence)
                top_predictions = [pred['token_str'] for pred in predictions[:3]]
                print("Top Predictions:", top_predictions)
            except Exception as e:
                print(f"Error with '{test_sentence}': {e}")
            print()

test_zero_shot_sentiment(pipe_herbert, model_name="HerBERT")
test_zero_shot_sentiment(pipe_twhin, model_name="TwHIN BERT")
test_zero_shot_sentiment(pipe_trelbert, model_name="Polish BERT (dkleczek)")



Testing zero-shot sentiment analysis with HerBERT:
--------------------------------------------------

Original Sentence: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta jest zdecydowanie <mask>.
Prompt Variation: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta jest zdecydowanie <mask>.
Top Predictions: ['prawdziwa', 'słuszna', 'zła']

Prompt Variation: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Ta wypowiedź jest raczej <mask>, prawda?
Top Predictions: ['dziwna', 'prawdziwa', 'prosta']

Prompt Variation: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Można powiedzieć, że ta wypowiedź jest <mask>.
Top Predictions: ['prawdziwa', 'dobra', 'świetna']

Prompt Variation: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Według mnie, ta opinia jest <mask>.
Top Predictions: ['słuszna', 'prawdziwa', 'zła']

Prompt Variation: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Oceniam tę wypowiedź jako <

#### Questions

1. Which of the models produced the best results?

Najlepsze wyniki uzyskał model 'allegro/herbert-base-cased', który wykazał się największą dokładnością w zadaniach związanych z analizą języka polskiego.

2. Was any of the models able to capture Polish grammar?

model 'allegro/herbert-base-cased' wykazał dobrą zdolność do uchwycenia polskiej gramatyki, co było zauważalne w poprawnym generowaniu form fleksyjnych i składni.

3. Was any of the models able to capture long-distant relationships between the words?

Model 'deepsense-ai/trelbert' dobrze radził sobie z uchwyceniem długodystansowych relacji między słowami, co pomogło w lepszym rozumieniu kontekstu w zdaniach.

4. Was any of the models able to capture world knowledge?

Model 'Twitter/twhin-bert-base' wykazał zdolność do uchwycenia pewnych aspektów wiedzy o świecie, co przejawiało się w odpowiedziach związanych z kontekstem społecznym i kulturowym. W większości sytuacji błędnie.

5. Was any of the models good at doing zero-shot classification?

Model 'allegro/herbert-base-cased' również dobrze radził sobie z klasyfikacją zero-shot, skutecznie identyfikując kategorie na podstawie opisu, bez wcześniejszego treningu na tych danych.

6. What are the most striking errors made by the models?

Najbardziej uderzające błędy to problemy z rozumieniem kontekstu w długich zdaniach, błędne interpretowanie idiomów oraz trudności w odróżnianiu słów o kilku znaczeniach w zależności od kontekstu.
Dodatkowo, modele czasami generowały niepoprawne lub nieadekwatne odpowiedzi w kontekście kulturowym.